In [4]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/trading
Changed current working directory to: /Users/falcon/Developments/trading
Initialized project with base directory: /Users/falcon/Developments/trading


In [18]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
top_60_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(60)
top_60_coins_df = top_60_coins_df.reset_index(drop=True)
top_60_coins = top_60_coins_df.get('name')

In [19]:
from constants.coin_names import CoinNames
from settings import DATA_DIR
import pandas as pd
import os
dfs = {}
no_data = []
# for coin in CoinNames.__members__:
for coin in top_60_coins:
    p = DATA_DIR / f'{coin}.csv'
    if os.path.exists(p):
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        dfs[coin] = df
    else:
        no_data.append(coin)
else:
    os.sys.stderr.write(f'No data for {no_data}')

No data for []

In [20]:
dfs["SOL"].iloc[-1, :]["tic"]

1710115200000

In [21]:
from collections import Counter


def get_majority_shape(dfs):
    # Get the shapes of all DataFrames
    shapes = [df.shape for df in dfs.values()]
    
    # Count occurrences of each shape
    shape_counts = Counter(shapes)
    
    # Display votes for each shape
    for shape, count in shape_counts.items():
        print(f"Shape: {shape}, Votes: {count}")
    
    # Find the most common shape
    majority_shape, majority_votes = shape_counts.most_common(1)[0]
    
    return majority_shape

def get_majority_start_end(dfs):
    # Get the shapes of all DataFrames
    start_end = [(df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"] ) for df in dfs.values()]
    
    # Count occurrences of each shape
    start_end_counts = Counter(start_end)
    
    # Display votes for each shape
    for se, count in start_end_counts.items():
        print(f"Start End: {se}, Votes: {count}")
    
    # Find the most common shape
    majority_se, majority_votes = start_end_counts.most_common(1)[0]
    
    return majority_se

major_shape = get_majority_shape(dfs)
major_se = get_majority_start_end(dfs)

Shape: (2881, 11), Votes: 57
Shape: (1963, 11), Votes: 2
Shape: (2441, 11), Votes: 1
Start End: (1707523200000, 1710115200000), Votes: 57
Start End: (1708349400000, 1710115200000), Votes: 2
Start End: (1707919200000, 1710115200000), Votes: 1


In [22]:
def filter_dfs(dfs, shape, start_end):
    valid_dfs = {}
    for coin, df in dfs.items():
        if df.shape != shape:
            print(f"Shape mismatch for {coin}. Expected {shape}, got {df.shape}")
            continue
        elif (df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"]) != start_end:
            print(f"Start/End mismatch for {coin}. Expected {start_end}, got {df.iloc[0, :]['tic'], df.iloc[-1, :]['tic']}")
            continue
        else:
            valid_dfs[coin] = df

    return valid_dfs


valid_dfs = filter_dfs(dfs, major_shape, major_se)

Shape mismatch for PIXEL. Expected (2881, 11), got (1963, 11)
Shape mismatch for STRK. Expected (2881, 11), got (2441, 11)
Shape mismatch for TAO. Expected (2881, 11), got (1963, 11)


In [23]:
df = valid_dfs["SOL"]

df.head()

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,
SOL,2024-02-10-00:00:00,2024-02-10-00:14:59,15m,365,107.63,107.66,106.98,107.03,12256.22,1707523200000,1707524099999
SOL,2024-02-10-00:15:00,2024-02-10-00:29:59,15m,240,107.71,108.03,107.51,107.66,7611.01,1707524100000,1707524999999
SOL,2024-02-10-00:30:00,2024-02-10-00:44:59,15m,269,107.56,108.29,107.36,107.79,18102.60,1707525000000,1707525899999
SOL,2024-02-10-00:45:00,2024-02-10-00:59:59,15m,149,107.67,107.83,107.26,107.57,3519.97,1707525900000,1707526799999
SOL,2024-02-10-01:00:00,2024-02-10-01:14:59,15m,225,107.55,108.09,107.48,107.73,7679.26,1707526800000,1707527699999


In [24]:
_df = pd.concat(valid_dfs.values())
_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164217 entries, BTC to LDO
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   start_time        164217 non-null  object 
 1   end_time          164217 non-null  object 
 2   interval          164217 non-null  object 
 3   number_of_trades  164217 non-null  int64  
 4   close             164217 non-null  float64
 5   high              164217 non-null  float64
 6   low               164217 non-null  float64
 7   open              164217 non-null  float64
 8   volume            164217 non-null  float64
 9   tic               164217 non-null  int64  
 10  toc               164217 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 15.0+ MB


In [25]:
_df.head()

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,
BTC,2024-02-10-00:00:00,2024-02-10-00:14:59,15m,1258,47244.0,47278.0,47160.0,47166.0,24.81303,1707523200000,1707524099999
BTC,2024-02-10-00:15:00,2024-02-10-00:29:59,15m,1099,47204.0,47283.0,47186.0,47244.0,23.62129,1707524100000,1707524999999
BTC,2024-02-10-00:30:00,2024-02-10-00:44:59,15m,1429,47147.0,47232.0,47077.0,47204.0,39.37283,1707525000000,1707525899999
BTC,2024-02-10-00:45:00,2024-02-10-00:59:59,15m,1048,47116.0,47153.0,47079.0,47146.0,28.94293,1707525900000,1707526799999
BTC,2024-02-10-01:00:00,2024-02-10-01:14:59,15m,993,47110.0,47158.0,47082.0,47122.0,28.31493,1707526800000,1707527699999


In [26]:
import numpy as np


df = _df
############ pct_change
df["pct_change"] = df.groupby(level=0)["close"].pct_change() * 100

############ accumulated_pct_change
df['multiplier'] = 1 + (df['pct_change'] / 100)
df['accumulated_multiplier'] = df.groupby(level=0)['multiplier'].cumprod()
df['accumulated_pct_change'] = (df['accumulated_multiplier'] - 1) * 100
df.drop(columns=['multiplier', 'accumulated_multiplier'], inplace=True)

############ rolling_accumulated_pct_change
df['start_time'] = pd.to_datetime(df['start_time'])
df['multiplier'] = 1 + (df['pct_change'] / 100)
window_size = 7 * 24 * 4  # Adjust this based on your exact data frequency
df['rolling_accumulated_multiplier'] = df.groupby(level=0)['multiplier'].transform(
    lambda x: x.rolling(window=window_size, min_periods=1).apply(np.prod)
)
df['rolling_accumulated_pct_change'] = (df['rolling_accumulated_multiplier'] - 1) * 100
df.drop(columns=['multiplier', 'rolling_accumulated_multiplier'], inplace=True)



df

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc,pct_change,accumulated_pct_change,rolling_accumulated_pct_change
symbol,,,,,,,,,,,,,,
BTC,2024-02-10 00:00:00,2024-02-10-00:14:59,15m,1258,47244.0000,47278.0000,47160.0000,47166.0000,24.81303,1707523200000,1707524099999,NaN,NaN,NaN
BTC,2024-02-10 00:15:00,2024-02-10-00:29:59,15m,1099,47204.0000,47283.0000,47186.0000,47244.0000,23.62129,1707524100000,1707524999999,-0.084667,-0.084667,-0.084667
BTC,2024-02-10 00:30:00,2024-02-10-00:44:59,15m,1429,47147.0000,47232.0000,47077.0000,47204.0000,39.37283,1707525000000,1707525899999,-0.120752,-0.205317,-0.205317
BTC,2024-02-10 00:45:00,2024-02-10-00:59:59,15m,1048,47116.0000,47153.0000,47079.0000,47146.0000,28.94293,1707525900000,1707526799999,-0.065752,-0.270934,-0.270934
BTC,2024-02-10 01:00:00,2024-02-10-01:14:59,15m,993,47110.0000,47158.0000,47082.0000,47122.0000,28.31493,1707526800000,1707527699999,-0.012735,-0.283634,-0.283634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LDO,2024-03-10 23:00:00,2024-03-10-23:14:59,15m,51,3.2533,3.2542,3.2205,3.2205,9344.20000,1710111600000,1710112499999,1.040437,8.671544,-2.560800
LDO,2024-03-10 23:15:00,2024-03-10-23:29:59,15m,20,3.2629,3.2629,3.2448,3.2484,6651.10000,1710112500000,1710113399999,0.295085,8.992217,-2.073830
LDO,2024-03-10 23:30:00,2024-03-10-23:44:59,15m,22,3.2740,3.2740,3.2563,3.2665,1128.30000,1710113400000,1710114299999,0.340188,9.362996,-1.604857


In [27]:
df.to_csv(DATA_DIR / "all.csv", index=True)